# Section 4 :  RecSys Model with TuriCreate

 - In this section, I will be working on an Item-based collaborative filtering using turicreate using the following sub-tasks;
 
- Transforming and normalizing data


- Training models

  
- Evaluating model performance

  
- Selecting the optimal model


In [185]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
from pandas.api.types import CategoricalDtype
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
#pd.options.plotting.backend = 'plotly'

###################################################
import random
import scipy.sparse as sparse
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve


########################################################
from itertools import islice
from itertools import cycle
#######################################################
import turicreate as tc


In [3]:
recsys_df = pd.read_csv('Olaleke_Tasks/RecSys_df.csv')
recsys_df.head()

,CardHolder,Material,SalesCount,ItemPrice,net_weight,SubCategorycode,ItemDescription,Class,rpa_wgh2,rpa_wgh2_SubClass,rpa_wgh3,rpa_wgh3_SubClass,vendor
0,10008254,109900,3.0,20.97,23.0,401201,Lenta bags,NON FOOD,40.0,"""Household cleaning products """,4012.0,Packaging and containers,100260.0
1,10011935,109900,1.0,6.49,23.0,401201,Lenta bags,NON FOOD,40.0,"""Household cleaning products """,4012.0,Packaging and containers,100260.0
2,10036221,109900,3.0,19.47,23.0,401201,Lenta bags,NON FOOD,40.0,"""Household cleaning products """,4012.0,Packaging and containers,100260.0
3,10036221,109900,2.0,12.98,23.0,401201,Lenta bags,NON FOOD,40.0,"""Household cleaning products """,4012.0,Packaging and containers,100260.0
4,10036221,109900,4.0,25.96,23.0,401201,Lenta bags,NON FOOD,40.0,"""Household cleaning products """,4012.0,Packaging and containers,100260.0


In [4]:
No_lentabag_df = pd.read_csv('Olaleke_Tasks/No_lentabag.csv')
No_lentabag_df.head()

,CardHolder,Material,SalesCount,ItemPrice,net_weight,SubCategorycode,ItemDescription,Class,rpa_wgh2,rpa_wgh2_SubClass,rpa_wgh3,rpa_wgh3_SubClass,vendor
0,10008254,439630,1.0,169.99,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0
1,14919617,439630,1.0,109.89,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0
2,15041309,439630,1.0,169.99,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0
3,15041309,439630,1.0,116.99,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0
4,15041309,439630,1.0,152.14,500.0,420103,Dry dog food up to 800 gr,FOOD,42.0,Pet goods,4201.0,Goods for dogs,100410.0


In [187]:
#No_lentabag_df.head()

## Creating Different Data Transformation form for Training

- 1.Data set with quantity of items boought    


In [6]:
user_item_df = pd.read_csv('Olaleke_Tasks/user_item_nonbag.csv')
user_item_df.head()

,CardHolder,Material,SalesCount
0,10008254,4135,2.0
1,10008254,5383,2.0
2,10008254,5880,2.0
3,10008254,78790,2.0
4,10008254,81107,1.0


In [7]:
df = user_item_df

In [8]:
item_lookup = pd.read_csv('Olaleke_Tasks/item_lookup_nonbag.csv')
item_lookup.head()

,Material,ItemDescription
0,439630,Dry dog food up to 800 gr
1,338329,Dry dog food up to 800 gr
2,338283,Dry dog food up to 800 gr
3,338287,Dry dog food up to 800 gr
4,513215,Dry dog food up to 800 gr


- 2.Data set with "purchase mask" that tells if a customers bought an item or not

In [9]:
            #Data form that only tells if a customers bought an item or not             
            #...same number of rows as before, we only added a new column...
def purchase_mask(df):
    df_dummy = df.copy()
    df_dummy['purchase_or_not_purpchase'] = 1
    return df_dummy

df_purchase_mask = purchase_mask(df)
df_purchase_mask.head()

,CardHolder,Material,SalesCount,purchase_or_not_purpchase
0,10008254,4135,2.0,1
1,10008254,5383,2.0,1
2,10008254,5880,2.0,1
3,10008254,78790,2.0,1
4,10008254,81107,1.0,1



- 3.Data with normalized items (by purchase frequency) across all users;


    - To do this, we normalize purchase frequency of each item across users bu creating a user-item matrix
    

In [266]:
                           #for each user.....
                           #we have a matrix with the number of item quantitis as matrix values 
df_matrix = pd.pivot_table(df,values='SalesCount',
                              index='CardHolder',
                              columns = 'Material')
      
                          #normalization of the df_matrix 
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max() - df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()    

(86497, 58320)


Material,21,85,94,602,613,616,627,655,684,729,...,532164,532198,532204,532225,532249,532250,532251,532291,532292,532293
CardHolder,,,,,,,,,,,,,,,,,,,,,
10008254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10011587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10011935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10036221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10049978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [267]:
df_norm_purchase = df_matrix_norm.reset_index()
df_norm_purchase.index_names = ['scaled_purchase_freq']

In [ ]:
df_norm_purchase = pd.melt(df_norm_purchase,
                       id_vars=['CardHolder'],
                       value_name = 'scaled_purchase_freq').dropna()

print(df_norm_purchase.shape)
df_norm_purchase.head()                             

In [12]:
df_norm_purchase.to_csv('scaled_purchase_freq.csv', index=False)

In [10]:
          #splitting datasets to train and test set :                                       
def split_data(df):
    train,test = train_test_split(df,test_size=0.2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data,test_data

               
#train_df_norm,test_df_norm = split_data(df_norm_purchase)
df_purchase_mask_tr,df_purchase_mask_ts = split_data(df_purchase_mask)
user_item_tr,user_item_ts = split_data(user_item_df)

In [40]:
train_df_norm

CardHolder,Material,scaled_purchase_freq
82732233,500016,0.0
82790682,428426,0.0
82908725,69267,0.038461538461538464
82731316,9845,0.0007861635220125787
61137360,498712,0.0
82959680,41422,0.0015878914828203652
59145723,338232,0.02631578947368421
48815084,345118,0.0
82790063,427309,0.0
82733361,11230,0.0001300483470606588


## Creating Diferent Turicreate Model Prototypes 

In [ ]:
                #

In [133]:
            #defining functions for model selection...
def model_selection(train_data,model_name,user_id,item_id,target,
                    users_to_recommend,n_rec=5,item_lookup=item_lookup,n_display=30):
    if model_name == 'popularity':
        model = tc.popularity_recommender.create(
                                      train_data,
                                      user_id = user_id,
                                      item_id = item_id,
                                      target= target)
    elif model_name == 'cosine':               #model based on item similarity using 'COSINE'
        model = tc.item_similarity_recommender.create(
                                      train_data,
                                      user_id = user_id,
                                      item_id = item_id,
                                      target= target,
                                      similarity_type = 'cosine')  
    elif model_name == 'pearson':               #model based on item similarity using 'PEARSON'
        model = tc.item_similarity_recommender.create(
                                      train_data,
                                      user_id = user_id,
                                      item_id = item_id,
                                      target= target,
                                      similarity_type = 'pearson')  

    elif model_name == 'jaccard':           #model based on item similarity using 'JACCARD'
        model = tc.item_similarity_recommender.create(
                                      train_data,
                                      user_id = user_id,
                                      item_id = item_id,
                                      target= target,
                                      similarity_type = 'jaccard')
    
                                            #making recommendation based on the the most popular items....
    recommendation = model.recommend(users = users_to_recommend,k = n_rec)
    clear_output()
    #recommendation.print_rows(n_display)
    
    rec_df  =  recommendation.to_dataframe()
    material_id = list(rec_df['Material'].unique()) 
    get_item = item_lookup.loc[item_lookup['Material'].isin(material_id)]
    get_item = get_item.reset_index(drop=True)
    
    rec_items = pd.merge(rec_df,get_item,
                        on='Material',how='left')  #preserve the sort order from rec_sample if preserved by "how='left' "
    rec_items = rec_items[['CardHolder','Material','ItemDescription','score','rank']]
    
    return model,rec_items


### Popularity Model

- Using the Popularity model as a baseline:

      
    - This takes the most popular items for recommendations
    
    - these items are the product with the highest number of sells across the customers
  

- Popularity Model based on Quantity of items

In [164]:
        #Popularity Model based on Quantity of items  
user_id = 'CardHolder'
item_id = 'Material'
target =  'SalesCount'
users_to_recommend = list(user_item_df[user_id].unique())
n_rec = 10                    #..number of items to recommend
n_display = 30                #..num of rows to display     

In [110]:
len(users_to_recommend)

86497

In [111]:

            ##testing model...
            ##popularity model...
model_name = 'popularity'
target = 'SalesCount'
popularity_model_qqt, pop_qqt_rec = model_selection(user_item_tr,model_name,user_id,item_id, target,users_to_recommend)
pop_qqt_rec[:n_display]

,CardHolder,Material,ItemDescription,score,rank
0,10008254,512637,"""Non-alcohol beer """,48.000000,1
1,10008254,161810,Premium and imported water,39.000000,2
2,10008254,378667,Individual portion food (pouch),20.854093,3
3,10008254,529747,Other long-term storage seasonal products,17.500000,4
4,10008254,482424,Licensed beer in can,16.506130,5
5,10011587,512637,"""Non-alcohol beer """,48.000000,1
6,10011587,161810,Premium and imported water,39.000000,2
7,10011587,378667,Individual portion food (pouch),20.854093,3
8,10011587,529747,Other long-term storage seasonal products,17.500000,4
9,10011587,482424,Licensed beer in can,16.506130,5


In [165]:
model_name = 'popularity'
target = 'purchase_or_not_purpchase'
popularity_model_qqt2,pop_mask_rec = model_selection(df_purchase_mask_tr,model_name,user_id,item_id, target,users_to_recommend)
pop_mask_rec[:n_display]

,CardHolder,Material,ItemDescription,score,rank
0,10008254,8048,"Baking foil, sleeve",1.0,1
1,10008254,387938,"""Flavoured green tea bags """,1.0,2
2,10008254,302231,Fish and meat snacks,1.0,3
3,10008254,438448,"""Milk chocolate bars """,1.0,4
4,10008254,139201,"Chicken eggs, 9-15 eggs",1.0,5
5,10011587,8048,"Baking foil, sleeve",1.0,1
6,10011587,387938,"""Flavoured green tea bags """,1.0,2
7,10011587,302231,Fish and meat snacks,1.0,3
8,10011587,438448,"""Milk chocolate bars """,1.0,4
9,10011587,139201,"Chicken eggs, 9-15 eggs",1.0,5


In [58]:
model_name = 'popularity'
target = 'scaled_purchase_freq'
popularity_model_norm,pop_norm_rec = model_selection(train_df_norm,model_name,user_id,item_id, target,users_to_recommend) 
pop_norm_rec[:n_display]

+------------+----------+-------+------+
| CardHolder | Material | score | rank |
+------------+----------+-------+------+
|  10008254  |  517191  |  1.0  |  1   |
|  10008254  |  389771  |  1.0  |  2   |
|  10008254  |  450267  |  1.0  |  3   |
|  10008254  |  525925  |  1.0  |  4   |
|  10008254  |  278577  |  1.0  |  5   |
|  10008254  |  517191  |  1.0  |  1   |
|  10008254  |  389771  |  1.0  |  2   |
|  10008254  |  450267  |  1.0  |  3   |
|  10008254  |  525925  |  1.0  |  4   |
|  10008254  |  278577  |  1.0  |  5   |
|  10008254  |  517191  |  1.0  |  1   |
|  10008254  |  389771  |  1.0  |  2   |
|  10008254  |  450267  |  1.0  |  3   |
|  10008254  |  525925  |  1.0  |  4   |
|  10008254  |  278577  |  1.0  |  5   |
|  10008254  |  517191  |  1.0  |  1   |
|  10008254  |  389771  |  1.0  |  2   |
|  10008254  |  450267  |  1.0  |  3   |
|  10008254  |  525925  |  1.0  |  4   |
|  10008254  |  278577  |  1.0  |  5   |
|  10008254  |  517191  |  1.0  |  1   |
|  10008254  |  

In [166]:
df_purchase_info_tr = df_purchase_mask_tr

In [168]:
            ##testing model...
            ##collaborative filtering model purchase_or_not_purpchase...
model_name = 'cosine'
target = 'purchase_or_not_purpchase'
cosine_model_mask1,cosine_mask_rec = model_selection(df_purchase_info_tr,model_name,user_id,item_id, target,users_to_recommend)
cosine_mask_rec[:n_display]

,CardHolder,Material,ItemDescription,score,rank
0,10008254,303640,Bulb onion,0.113423,1
1,10008254,11230,Bananas,0.113356,2
2,10008254,94429,Lemons,0.104768,3
3,10008254,303639,Carrot,0.101521,4
4,10008254,11189,"""Cabbage """,0.100215,5
5,10011587,11230,Bananas,0.106039,1
6,10011587,94429,Lemons,0.101006,2
7,10011587,303640,Bulb onion,0.099016,3
8,10011587,41424,Carrot,0.098907,4
9,10011587,216368,Packaged cucumbers,0.098078,5


In [ ]:
#train_df_norm,test_df_norm df_purchase_mask_tr,df_purchase_mask_ts 

In [48]:
            ##testing model...
            ##cosine model scaled_purchase_freq...
model_name = 'cosine'
target = 'scaled_purchase_freq'
cosine_model_norm,cosine_norm_rec = model_selection(train_df_norm,model_name,user_id,item_id, target,users_to_recommend)
cosine_norm_rec[:n_display]

+------------+----------+-----------------------+------+
| CardHolder | Material |         score         | rank |
+------------+----------+-----------------------+------+
|  10008254  |  486953  |  0.003813295260719631 |  1   |
|  10008254  |  487064  | 0.0023587449737217116 |  2   |
|  10008254  |  525902  |  0.002042713372603707 |  3   |
|  10008254  |  472179  | 0.0019066463346066682 |  4   |
|  10008254  |  170127  |  0.001797603524249533 |  5   |
|  10008254  |  486953  |  0.003813295260719631 |  1   |
|  10008254  |  487064  | 0.0023587449737217116 |  2   |
|  10008254  |  525902  |  0.002042713372603707 |  3   |
|  10008254  |  472179  | 0.0019066463346066682 |  4   |
|  10008254  |  170127  |  0.001797603524249533 |  5   |
|  10008254  |  486953  |  0.003813295260719631 |  1   |
|  10008254  |  487064  | 0.0023587449737217116 |  2   |
|  10008254  |  525902  |  0.002042713372603707 |  3   |
|  10008254  |  472179  | 0.0019066463346066682 |  4   |
|  10008254  |  170127  |  0.00

In [170]:
             ##testing model...
             ##cosine model quantity...
model_name = 'cosine'
target = 'SalesCount'
cosine_model_qqt,cosine_qqt_rec = model_selection(user_item_tr,model_name,user_id,item_id, target,users_to_recommend)
cosine_qqt_rec[:n_display]

,CardHolder,Material,ItemDescription,score,rank
0,10008254,457282,Other detergents,0.096228,1
1,10008254,449257,Perfume sets for men,0.081602,2
2,10008254,159200,Children's magazines,0.045716,3
3,10008254,469931,Mixed vegetable oil,0.044348,4
4,10008254,490118,"Garden lights, lanterns",0.043740,5
5,10011587,350426,Cream yoghurt for children older than 3 years,0.191514,1
6,10011587,355877,Cream yoghurt for children older than 3 years,0.182351,2
7,10011587,385030,Cream yoghurt for children older than 3 years,0.172070,3
8,10011587,504726,Cream yoghurt for children older than 3 years,0.162448,4
9,10011587,282741,"Curd rolls, snacks cold storage",0.148101,5


In [171]:
             ##testing model...
             ##jaccard model...
model_name = 'jaccard'
target = 'SalesCount'
jaccard_model_qqt,jaccard_qqt_rec = model_selection(user_item_tr,model_name,user_id,item_id, target,users_to_recommend)
jaccard_qqt_rec[:n_display]

,CardHolder,Material,ItemDescription,score,rank
0,10008254,45520,Bulk potato,0.032414,1
1,10008254,29221,"""Cabbage """,0.030996,2
2,10008254,41424,Carrot,0.030545,3
3,10008254,239792,Bulk traditional tomatoes,0.030119,4
4,10008254,39906,Marrows,0.029737,5
5,10011587,80424,Cow pasteurized milk <1 l,0.042652,1
6,10011587,503022,Oatmeal cookies,0.041633,2
7,10011587,492772,Oatmeal cookies,0.040752,3
8,10011587,135219,Natural curd 2%-5% fat,0.033361,4
9,10011587,114858,Packaged oranges,0.032940,5


In [51]:
#train_df_norm

In [53]:
            ##testing model...
            ##jaccard scaled_purchase_freq model...
model_name = 'jaccard'
target = 'scaled_purchase_freq'
jaccard_model_norm,jaccard_norm_rec = model_selection(train_df_norm,model_name,user_id,item_id, target,users_to_recommend)      
jaccard_norm_rec[:n_display]

+------------+----------+----------------------+------+
| CardHolder | Material |        score         | rank |
+------------+----------+----------------------+------+
|  10008254  |  11230   | 0.019119394862133522 |  1   |
|  10008254  |  303640  | 0.018552422523498535 |  2   |
|  10008254  |  94429   | 0.017335808795431385 |  3   |
|  10008254  |  41203   | 0.017288780730703602 |  4   |
|  10008254  |  45520   | 0.017197551934615425 |  5   |
|  10008254  |  11230   | 0.019119394862133522 |  1   |
|  10008254  |  303640  | 0.018552422523498535 |  2   |
|  10008254  |  94429   | 0.017335808795431385 |  3   |
|  10008254  |  41203   | 0.017288780730703602 |  4   |
|  10008254  |  45520   | 0.017197551934615425 |  5   |
|  10008254  |  11230   | 0.019119394862133522 |  1   |
|  10008254  |  303640  | 0.018552422523498535 |  2   |
|  10008254  |  94429   | 0.017335808795431385 |  3   |
|  10008254  |  41203   | 0.017288780730703602 |  4   |
|  10008254  |  45520   | 0.017197551934615425 |

In [172]:
            ##testing jaccard model...
            ##based on purchase masking...
model_name = 'jaccard'
target = 'purchase_or_not_purpchase'
jaccard_model_mask,jaccard_mask_rec = model_selection(df_purchase_info_tr,model_name,user_id,item_id, target,users_to_recommend)
jaccard_mask_rec[:n_display]

,CardHolder,Material,ItemDescription,score,rank
0,10008254,45520,Bulk potato,0.030796,1
1,10008254,29221,"""Cabbage """,0.029339,2
2,10008254,43027,"""Bulk apples """,0.028628,3
3,10008254,341482,Packaged cucumbers,0.028393,4
4,10008254,239792,Bulk traditional tomatoes,0.028290,5
5,10011587,492772,Oatmeal cookies,0.040683,1
6,10011587,503022,Oatmeal cookies,0.038507,2
7,10011587,184194,Sand sugar white,0.034949,3
8,10011587,80424,Cow pasteurized milk <1 l,0.029738,4
9,10011587,280164,"Chicken eggs, 9-15 eggs",0.027206,5


In [178]:
jaccard_qqt_rec.shape

(432485, 5)

In [179]:
jaccard_mask_rec.to_csv('jaccard_rec_purchase_mask.csv')
jaccard_qqt_rec.to_csv('jaccard_rec_salecount_qqt.csv')

In [219]:
#jaccard_mask_rec

### Evaluation of Models

In [180]:
              ##..
quantity_based_models = [popularity_model_qqt,
                         cosine_model_qqt,
                         jaccard_model_qqt]

mask_based_models = [popularity_model_qqt2,
                         cosine_model_mask1,
                         jaccard_model_mask]

# freq_based_models = [popularity_model_norm,
#                          cosine_model_norm,
#                          jaccard_model_norm]

              ##..
names_qqt = ['Popularity Model  on item quantity', 
              'Cosine Similarity on item quantity', 
              'Jaccard Similarity on item quantity']

names_mask =['Popularity Model  on purchase masking', 
            'Cosine Similarity on purchase masking', 
            'Jaccard Similarity on purchase masking']


name_freq = ['Popularity Model on purchase freq', 
              'Cosine Similarity on purchase freq', 
              'Jaccard Similarity on purchase freq']

In [181]:
tc.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 128)

In [ ]:
              ##..evaluation of models.
eval_quantity_based = tc.recommender.util.compare_models(user_item_ts,
                                                         quantity_based_models,
                                                          model_names = names_qqt )

PROGRESS: Evaluate model Popularity Model  on item quantity


recommendations finished on 1000/81697 queries. users per second: 33115.9

recommendations finished on 2000/81697 queries. users per second: 39572.6

recommendations finished on 3000/81697 queries. users per second: 42241

recommendations finished on 4000/81697 queries. users per second: 43534.1

...

recommendations finished on 78000/81697 queries. users per second: 36671

recommendations finished on 79000/81697 queries. users per second: 35476

recommendations finished on 80000/81697 queries. users per second: 35184.8

recommendations finished on 81000/81697 queries. users per second: 34542.9


Precision and recall summary statistics by cutoff
+--------+----------------------+-----------------------+
| cutoff |    mean_precision    |      mean_recall      |
+--------+----------------------+-----------------------+
|   1    | 0.06974552309142297  | 0.0028394419611409275 |
|   2    | 0.060222529591049866 | 0.0049136972056426625 |
|   3    | 0.05394322924954457  |  0.006609702652420212 |
|   4    | 0.049576483836615864 |  0.008015180215140654 |
|   5    | 0.04646927059745055  |  0.009177266621752046 |
|   6    | 0.04379189770330162  |  0.010289833149336684 |
|   7    | 0.041657413543775876 |  0.01142160360608337  |
|   8    | 0.039879065326756294 |  0.012383199683150218 |
|   9    | 0.038414303258791034 |  0.013332543556990945 |
|   10   | 0.03712498622960313  |  0.014159949091728864 |
+--------+----------------------+-----------------------+
[10 rows x 3 columns]



In [76]:
eval_mask_based = tc.recommender.util.compare_models(df_purchase_info_ts,
                                                          mask_based_models,
                                                           model_names = names_mask )

In [ ]:
eval_freq_based = tc.recommender.util.compare_models(test_df_norm,
                                                         freq_based_models,
                                                          model_names = name_freq)

PROGRESS: Evaluate model Popularity Model on purchase freq


recommendations finished on 1000/82045 queries. users per second: 15114

recommendations finished on 2000/82045 queries. users per second: 15877.9

recommendations finished on 3000/82045 queries. users per second: 20901

recommendations finished on 4000/82045 queries. users per second: 25368.2

...

recommendations finished on 79000/82045 queries. users per second: 72804.6

recommendations finished on 80000/82045 queries. users per second: 72029.1

recommendations finished on 81000/82045 queries. users per second: 70734.1

recommendations finished on 82000/82045 queries. users per second: 67280.1


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.006325796818818931 | 0.0005797958975769801 |
|   2    |  0.005655432994088612 | 0.0010296032627884171 |
|   3    |  0.005224775021837558 | 0.0013873421516376549 |
|   4    |  0.004841855079529527 |  0.001665457124625654 |
|   5    | 0.0046169784874153285 | 0.0020013951519831973 |
|   6    | 0.0044183070266317764 | 0.0022509944682372186 |
|   7    |  0.004241574745566542 | 0.0024640519537008684 |
|   8    |  0.004115119751355968 |  0.002759550334998517 |
|   9    |  0.003982909108145184 | 0.0029913991515556635 |
|   10   | 0.0039039551465659215 | 0.0032621413422985798 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]



### Recommendation Examples

In [11]:
CUSTOMERS = list(np.sort(df['CardHolder'].unique()))      #Get list of unique customers..
ITEMS     = list(df['Material'].unique())                 #Get list of unique items
QUANTITY  =  list(df['SalesCount'])                       #get all quantities
customers_arr = np.array(CUSTOMERS)                       #Array of unique customer IDs from the ratings matrix
items_arr =  np.array(ITEMS)                              #Array of unique product IDs from the ratings matrix

                    #Creating rows values corresponding to unique Customers
rows = df.CardHolder.astype(CategoricalDtype(categories=CUSTOMERS)).cat.codes
                     #Creating columns values corresponding to unique Items
cols = df.Material.astype(CategoricalDtype(categories=ITEMS)).cat.codes 


Rating_matrix = sparse.csr_matrix((QUANTITY, (rows,cols)),
                                     shape=(len(CUSTOMERS),len(ITEMS)))
Rating_matrix

<86497x58320 sparse matrix of type '<class 'numpy.float64'>'
	with 15205494 stored elements in Compressed Sparse Row format>

In [91]:

                                                            #get the items that have already been purchased by
                                                            #...a specific customer in the training set
def get_item_already_bought(train_set,customers_list,items_list,item_lookup,num_items=5):
    already_purchased= []
    for customer_id in customers_list:
        customers_index = np.where(customers_list == customer_id)[0][0]    #get the row index  of the customer_ID 
        customer_item_index = train_set[customers_index,:].nonzero()[1]    #get column indices of purchased items by the customer 
        Item_id = items_list[customer_item_index] 
        get_item = item_lookup.loc[item_lookup['Material'].isin(Item_id)]
        get_item =  get_item.reset_index(drop=True)                  
        #get_item  = pd.DataFrame(get_item)
                                             #set a bew column for the customer
        customer = cycle([str(customer_id)])                                                      
        get_item['Cardholder'] = [next(customer) for customers in range(len(get_item))]    
        already_purchased.append(get_item[:num_items])
        
    already_purchased_df = pd.concat(already_purchased)
    already_purchased_df = already_purchased_df[['Cardholder','Material','ItemDescription']]
    already_purchased_df = already_purchased_df.rename(columns={'ItemDescription':'ItemsAlreadyBought'})

    return already_purchased_df 

In [95]:
#
ItemsAlreadyBought_df = get_item_already_bought(Rating_matrix,customers_arr,items_arr,item_lookup,num_items=5)
ItemsAlreadyBought_df.head()

,Cardholder,Material,ItemsAlreadyBought
0,10008254,439630,Dry dog food up to 800 gr
1,10008254,429670,Dry food for cats from 450 gr to 1500 gr
2,10008254,152314,"""Special liquid washing detergent """
3,10008254,81107,"""Milk shakes warm storage """
4,10008254,366220,Hair dyes


In [96]:
ItemsAlreadyBought_df.to_csv('ItemsAlreadyBought_df.csv', index=False)

In [119]:
ItemsAlreadyBought_df = ItemsAlreadyBought_df.rename(columns={'AlreadyBoughtMaterial':'MaterialAlreadyBought',
                                                             'Cardholder':'CardHolder'})
ItemsAlreadyBought_df['CardHolder'].nunique()

86497

In [123]:
ItemsAlreadyBought_df['CardHolder'] = ItemsAlreadyBought_df['CardHolder'].astype(int)    
ItemsAlreadyBought_df.dtypes

CardHolder                int64
MaterialAlreadyBought     int64
ItemsAlreadyBought       object
dtype: object

In [ ]:
# train_df_norm,test_df_norm df_purchase_mask_tr,df_purchase_mask_ts 

In [224]:
jaccard_mask_rec.head()

,CardHolder,Material,ItemDescription,score,rank
0,10008254,45520,Bulk potato,0.030796,1
1,10008254,29221,"""Cabbage """,0.029339,2
2,10008254,43027,"""Bulk apples """,0.028628,3
3,10008254,341482,Packaged cucumbers,0.028393,4
4,10008254,239792,Bulk traditional tomatoes,0.028290,5


In [256]:
def proper_result(customer_id,Recommendation_df1,Recommendation_df2,
                  Rec_item1,Rec_item2,ItemsAlreadyBought_df=ItemsAlreadyBought_df):
    
    ItemsAlreadyBought = ItemsAlreadyBought_df[ItemsAlreadyBought_df['CardHolder'] == customer_id]
    ItemsAlreadyBought =  ItemsAlreadyBought.reset_index(drop=True)
    
    Recommendation1 = Recommendation_df1[Recommendation_df1['CardHolder'] == customer_id]
    Recommendation1 =  Recommendation1.reset_index(drop=True)
    
    Recommendation2 = Recommendation_df2[Recommendation_df2['CardHolder'] == customer_id]
    Recommendation2 =  Recommendation2.reset_index(drop=True)
   
    Result = pd.concat([Recommendation1[['CardHolder','ItemDescription','score','rank']],
                        Recommendation2[['ItemDescription','score']],
                        ItemsAlreadyBought['ItemsAlreadyBought']], 
                        ignore_index=True, axis=1)

    Result = Result.rename(columns={
                      0:"CardHolder",
                      1:Rec_item1,
                      2:"Score1",
                      3:"Rank",
                      4:Rec_item2,
                      5:"Score2",
                      6:"ItemsAlreadyBought"})

    Result = Result[['CardHolder','ItemsAlreadyBought',Rec_item1,'Score1',Rec_item2,'Score2','Rank']]
    return Result



# def proper_result1(customer_id,Recommendation_df1,#Recommendation_df2,
#                           ItemsAlreadyBought_df=ItemsAlreadyBought_df):
    
#     ItemsAlreadyBought = ItemsAlreadyBought_df[ItemsAlreadyBought_df['CardHolder'] == customer_id]
#     ItemsAlreadyBought =  ItemsAlreadyBought.reset_index(drop=True)
    
#     Recommendation1 = Recommendation_df1[Recommendation_df1['CardHolder'] == customer_id]
#     Recommendation1 =  Recommendation1.reset_index(drop=True)
    
# #     Recommendation2 = Recommendation_df2[Recommendation_df2['CardHolder'] == customer_id]
# #     Recommendation2 =  Recommendation2.reset_index(drop=True)
   
#     Result = pd.concat([Recommendation1[['CardHolder','ItemDescription','score','rank']],
#                ItemsAlreadyBought['ItemsAlreadyBought']], 
#                ignore_index=True, axis=1)

#     Result = Result.rename(columns={
#                       0:"CardHolder",
#                       1:"Recommended_Items",
#                       2:"Score",
#                       3:"Rank",
#                       4:"ItemsAlreadyBought"})

#     Result = Result[['CardHolder','ItemsAlreadyBought','Recommended_Items','Score','Rank']]
#     return Result

#proper_result1(83332008,jaccard_mask_rec)
#proper_result1(83332008,jaccard_qqt_rec)

In [261]:
users_to_recommend[-10:]

[83326249,
 83326911,
 83327390,
 83329160,
 83329286,
 83329684,
 83329792,
 83332008,
 83332870,
 83333056]

In [262]:
proper_result(83326249,jaccard_qqt_rec,jaccard_mask_rec,'Rec_Items_Jac_qqt','Rec_Items_Jac_mask')

,CardHolder,ItemsAlreadyBought,Rec_Items_Jac_qqt,Score1,Rec_Items_Jac_mask,Score2,Rank
0,83326249,Table games,Drinking yoghurt for children older than 3 years,0.066867,Drinking yoghurt for children older than 3 years,0.066358,1
1,83326249,Table games,Drinking yoghurt for children older than 3 years,0.045666,Drinking yoghurt for children older than 3 years,0.044141,2
2,83326249,"""Table and drinking still water """,Drinking yoghurt for children older than 3 years,0.037834,Table games,0.039510,3
3,83326249,Drinking yoghurt for children older than 3 years,Table games,0.034619,Drinking yoghurt for children older than 3 years,0.038872,4
4,83326249,NaN,Drinking yoghurt for children older than 3 years,0.026656,Drinking yoghurt for children older than 3 years,0.029606,5


In [265]:
proper_result(83332870,jaccard_qqt_rec,jaccard_mask_rec,'Rec_Items_Jac_qqt','Rec_Items_Jac_mask')

,CardHolder,ItemsAlreadyBought,Rec_Items_Jac_qqt,Score1,Rec_Items_Jac_mask,Score2,Rank
0,83332870,"""Poultry salads with mayonnaise, own productio...",Other fast food products own production,0.090520,"Meat salads with mayonnaise, own production",0.085389,1
1,83332870,"Energy drinks < 0,6 l",Other fast food products own production,0.086684,"""Fish and seafood salads with mayonnaise, own ...",0.073569,2
2,83332870,Other fast food products own production,"Energy drinks < 0,6 l",0.077766,"""Vegetable salads with vegetable oil, own prod...",0.071191,3
3,83332870,NaN,"Energy drinks < 0,6 l",0.062500,"""Fish and seafood salads with mayonnaise, own ...",0.070943,4
4,83332870,NaN,"Sandwiches, sandwiches, rolls in pita bread ow...",0.053910,"""Fish and seafood salads with mayonnaise, own ...",0.070658,5


In [264]:
proper_result(83326911,jaccard_qqt_rec,jaccard_mask_rec,'Rec_Items_Jac_qqt','Rec_Items_Jac_mask')

,CardHolder,ItemsAlreadyBought,Rec_Items_Jac_qqt,Score1,Rec_Items_Jac_mask,Score2,Rank
0,83326911,"""Liquid washing detergent """,Canned peas,0.017926,Marrows,0.021488,1
1,83326911,Other grains,"""Cabbage """,0.015525,"""Cabbage """,0.019892,2
2,83326911,Baby fabric conditioner,"""Carbonated wine drinks """,0.015295,Spring onion,0.019326,3
3,83326911,"""Smoked white fish slices in branded package """,Packaged potato,0.015217,Other grains,0.019191,4
4,83326911,"""Cabbage ""","""Cabbage """,0.015206,"""Cabbage """,0.018861,5


In [263]:
proper_result(83329160,jaccard_qqt_rec,jaccard_mask_rec,'Rec_Items_Jac_qqt','Rec_Items_Jac_mask')

,CardHolder,ItemsAlreadyBought,Rec_Items_Jac_qqt,Score1,Rec_Items_Jac_mask,Score2,Rank
0,83329160,Butter packing 125 gr - 300 gr,Bulk potato,0.044370,Bulk potato,0.056375,1
1,83329160,Pants,"""Cabbage """,0.042731,Sand sugar white,0.053723,2
2,83329160,"""Medicinal mineral water """,Sand sugar white,0.042592,"""Cabbage """,0.052865,3
3,83329160,"""Cables for mobile phones """,Carrot,0.041035,Carrot,0.052225,4
4,83329160,"""Caviar imitation in branded package ""","""Cabbage """,0.040848,Cow pasteurized milk <1 l,0.052019,5


In [255]:
proper_result(10054537,jaccard_qqt_rec,jaccard_mask_rec,'Rec_Items_Jac_qqt','Rec_Items_Jac_mask')

,CardHolder,ItemsAlreadyBought,Rec_Items_Jac_qqt,Score1,Rec_Items_Jac_mask,Score2,Rank
0,10054537,Instant mashed potato,Fish fillet-pieces proprietary packaging,0.048913,Cheese piece with colored mold,0.100515,1
1,10054537,Cheese piece with colored mold,Butter packing 125 gr - 300 gr,0.026316,Instant mashed potato,0.089224,2
2,10054537,Fish fillet branded packaging,"""Baking mix """,0.024691,Instant mashed potato,0.086022,3
3,10054537,NaN,Role-playing game kit,0.023256,Cheese piece with colored mold,0.058164,4
4,10054537,NaN,Fish fillet-pieces proprietary packaging,0.021898,Cheese piece with colored mold,0.046929,5


In [257]:
proper_result(10071527,jaccard_qqt_rec,jaccard_mask_rec,'Rec_Items_Jac_qqt','Rec_Items_Jac_mask')

,CardHolder,ItemsAlreadyBought,Rec_Items_Jac_qqt,Score1,Rec_Items_Jac_mask,Score2,Rank
0,10071527,Cotton wool,Bulk potato,0.023795,Bulk potato,0.026887,1
1,10071527,Cotton wool,Marrows,0.021455,Spring onion,0.023560,2
2,10071527,"Baking foil, sleeve","""Cabbage """,0.021313,Bulk cucumbers,0.023349,3
3,10071527,"Washing detergent - capsules, tablets","""Cabbage """,0.021071,Marrows,0.023252,4
4,10071527,Cow pasteurized milk <1 l,Carrot,0.020751,Packaged cucumbers,0.022716,5


In [259]:
proper_result(10011587,jaccard_qqt_rec,jaccard_mask_rec,'Rec_Items_Jac_qqt','Rec_Items_Jac_mask')

,CardHolder,ItemsAlreadyBought,Rec_Items_Jac_qqt,Score1,Rec_Items_Jac_mask,Score2,Rank
0,10011587,"""Sparkling white wine, brut, dry, semi-dry """,Cow pasteurized milk <1 l,0.042652,Oatmeal cookies,0.040683,1
1,10011587,Cow pasteurized milk <1 l,Oatmeal cookies,0.041633,Oatmeal cookies,0.038507,2
2,10011587,Oatmeal cookies,Oatmeal cookies,0.040752,Sand sugar white,0.034949,3
3,10011587,Oatmeal cookies,Natural curd 2%-5% fat,0.033361,Cow pasteurized milk <1 l,0.029738,4
4,10011587,NaN,Packaged oranges,0.032940,"Chicken eggs, 9-15 eggs",0.027206,5


In [258]:
proper_result(10069565,jaccard_qqt_rec,jaccard_mask_rec,'Rec_Items_Jac_qqt','Rec_Items_Jac_mask')

,CardHolder,ItemsAlreadyBought,Rec_Items_Jac_qqt,Score1,Rec_Items_Jac_mask,Score2,Rank
0,10069565,"""Special liquid washing detergent """,Bulk potato,0.045760,Lemons,0.045794,1
1,10069565,Hair dyes,"""Cabbage """,0.043960,Bulk traditional tomatoes,0.045302,2
2,10069565,Hair dyes,Carrot,0.042591,"""Cabbage """,0.043726,3
3,10069565,Butter packing 125 gr - 300 gr,Lemons,0.041882,Bulk traditional tomatoes,0.043051,4
4,10069565,"Cut salmon, service",Bulk traditional tomatoes,0.040711,Sand sugar white,0.042420,5


In [260]:
proper_result(10049978,jaccard_qqt_rec,jaccard_mask_rec,'Rec_Items_Jac_qqt','Rec_Items_Jac_mask')

,CardHolder,ItemsAlreadyBought,Rec_Items_Jac_qqt,Score1,Rec_Items_Jac_mask,Score2,Rank
0,10049978,Bananas,Bulb onion,0.084166,Bulb onion,0.067884,1
1,10049978,Condensed milk,"""Sweet pepper """,0.076731,"""Sweet pepper """,0.061522,2
2,10049978,Lemons,Carrot,0.071639,Carrot,0.057303,3
3,10049978,Premium and imported water,Bulk tangerines,0.067433,Bulk tangerines,0.053680,4
4,10049978,Hazelnut,Bulk cucumbers,0.064440,Carrot,0.051713,5
